# Değişken Kapsamı ve Bağlayıcılığı

## Yerel Olmayan Değişkenler (nonlocal)

Python 3, yerel olmayan adlı yeni bir anahtar kelime ekledi. Yerel olmayan anahtar sözcük, iç kapsama bir kapsam geçersiz kılma ekler.
Bununla ilgili her şeyi <a href="https://www.python.org/dev/peps/pep-3104/"> PEP 3104</a>'te okuyabilirsiniz. Bu, en iyi birkaç kod örneği ile açıklanmaktadır. En yaygın örneklerden biri, artabilen işlev oluşturmaktır:

In [1]:
def counter():
    num = 0
    def get_increse():
        num += 1
        return num
    return get_increse

number = counter()
print(number())

UnboundLocalError: cannot access local variable 'num' where it is not associated with a value

Bu kodu çalıştırmayı denerseniz, bir UnboundLocalError alacaksınız çünkü num değişkeni en içteki işleve atanmadan önce başvurulmaktadır. Karışıma yerel olmayan ekleyelim:

In [2]:
def counter():
    num = 0
    def get_increse():
        nonlocal num
        num += 1
        return num
    return get_increse

number = counter()
print(number())

1


Temelde nonlocal, bir dış kapsamdaki değişkenlere atamanıza izin verir, ancak genel bir kapsamda atamanıza izin vermez. Dolayısıyla, sayici fonksiyonumuzda yerel olmayan kullanamazsınız çünkü o zaman global bir kapsama atamaya çalışır. Syntax eror hatası verir.

In [3]:
def counter():
    nonlocal num # Hataya neden olur. nonlocal genel bir kapsamda atamanıza izin vermez.
    num = 0
    def get_increse():        
        num += 1
        return num
    return get_increse

number = counter()
print(number())

SyntaxError: no binding for nonlocal 'num' found (3460029626.py, line 2)

## Global Değişkenler (Global Variables)

Python'da, fonksiyonların içindeki değişkenler yerel olarak kabul edilirler, ancak ve ancak bir atama ifadesinin sol tarafında veya başka bir bağlama oluşumunda görünürlerse; aksi takdirde böyle bir bağlanma, genel kapsama kadar kapalı fonksiyonlarda aranır. Atama ifadesi asla çalıştırılmasa bile bu doğrudur.

In [4]:
x = 'Hi'
def read_2_x():
    print(x) # x sadece referans gösterildi, bu nedenle global kabul edildi
read_2_x()

Hi


In [5]:
y = "sss"
def read_2_y():
    y = "kkk"
    print(y) # Sadece y referans alınmıştır, bu nedenle global kabul edilir
read_2_y() # NameError: genel adı 'y' tanımlı değildir
print(y)

kkk
sss


In [6]:
def read_2_y():
    y = 'Hey' # y yereldir
    print(y) # yerel y'yi bulacak
read_2_y() # Hey yazdırır

Hey


In [23]:
print(y) # y globalde olmadığı için Name Error Verir

NameError: name 'y' is not defined

In [7]:
def read_m_local_fail():
    if True:
        m = 'Hey' # m yereldir
    return m
# print(m) # atanmamış _local_ m'yi arayacak ve bulunamayacak

print(read_m_local_fail())

Hey


Normalde, bir kapsam içindeki bir atama, aynı ada sahip tüm dış değişkenleri gölgeleyecektir:

In [5]:
x = 'Hi'
def change_local_x():    
    x = 'Bye'
    print(x)
    def change_local():
        global y
        x = "Hello"
        y = "Merhaba"
        print(x)
    change_local()
    print(x)

print(x)
change_local_x() # prints Bye
print(x) # prints Hi
print(y)

#Hi
#Bye
#Hello
#Hello
#Hi

Hi
Bye
Hello
Bye
Hi
Merhaba


Bir adın **global** olarak bildirilmesi, kapsamın geri kalanı için adla ilgili herhangi bir atamanın modülün en üst düzeyinde gerçekleşeceği anlamına gelir:

In [24]:
x = 'Hi'
def change_global_x():
    global x
    x = 'Bye'
    print(x)
change_global_x() # Bye yazdırır
print(x) # Bye yazdırır

Bye
Bye


Global anahtar sözcüğü, atamaların programın en üst düzeyinde değil, modülün en üst düzeyinde olacağı anlamına gelir.
Diğer modüller, modül içindeki değişkenlere her zamanki noktalı erişime ihtiyaç duyacaktır.
Özetlemek gerekirse: x değişkeninin bir fonksiyon için yerel olup olmadığını bilmek için, fonksiyonun tamamını okumalısınız:
1. global x bulduysanız, x global bir değişkendir
2. Yerel olmayan x bulduysanız, x kapsayıcı bir işleve aittir ve ne yerel ne de küreseldir
3. x = 5 veya aralık (3) 'te x veya başka bir bağlama bulduysanız, x yerel bir değişkendir
4. Aksi takdirde, x bazı çevreleyen kapsamlara aittir (işlev kapsamı, genel kapsam veya yerleşikler)

## Yerel Değişkenler

Bir değişken bir fonksiyonun içine bağlıysa, varsayılan olarak yalnızca işlev içinden erişilebilir:

In [7]:
def foo():
    a = 5
    print(a) # sorun yok
foo() # NameError: name 'a' is not defined

5


Kontrol akışı yapılarının kapsam üzerinde hiçbir etkisi yoktur (hariç hariç), ancak henüz atanmamış değişkene erişmek bir hatadır:

In [10]:
def foo():
    if True:
        a = 5
    print(a) # ok
b = 3
def bar():
    b = None
    if False:
        b = 5
    print(b) 
bar() # UnboundLocalError: local variable 'b' referenced before assignment

None


## if-while değişkenleri

**if ve while** bloklarında tanımlanan değişkenler **yerel bir değişken yerine global bir değişken** olmaktadır.

In [1]:
if True:
    t = 10
    print(t)

print(t)

10
10


In [2]:
while True:
    deger =  10
    print(deger)
    break

print(deger)

10
10


In [3]:
x = 5
y = 3
z = 2
def toplama(x,y,z):
    return x+y+z

print(toplama(x,y,z))

10


In [1]:
for i in range(10):
    y = 0
    y += i
    
print(y)

9


In [2]:
count = 0
while count < 10:
    y = 0
    y += count
    count += 1
    
print(y)

9


### Tüm kavramları birleştirelim

In [2]:
x = "global scope"
def outer_func():
    x = "outer func scope"
    def local_func():
        x = "local func scope"
        print("local_worked:", x)
        
    def nonlocal_func():
        nonlocal x
        x = "outer func scope changed"
        print("nonlocal_worked:", x)

    def global_func():
        global x
        x = "global scope changed"
        print("global_worked:", x)

    local_func()
    print("After local_func:", x)
    nonlocal_func()
    print("After nonlocal_func:", x)
    global_func()
    print("After global_func:", x)
    
print(x)
outer_func()
print("Global:", x)

global scope
local_worked: local func scope
After local_func: outer func scope
nonlocal_worked: outer func scope changed
After nonlocal_func: outer func scope changed
global_worked: global scope changed
After global_func: outer func scope changed
Global: global scope changed
